In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: johnwilsonlorin1 (johnwilsonlorin1-laguna-state-polytechnic-university). Use `wandb login --relogin` to force relogin


True

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import wandb

# Initialize a new wandb run
wandb.init(project='LLDA-Detecting Algal Blooms and Fish Kill', name = "RandomForestRegressor")

# Load datasets with specified encoding
merged_df = pd.read_csv('New_Merged.csv')
merged_df.dropna(inplace=True)

# Select relevant features and target
features = ['Temperature', 'Humidity', 'Wind', 'Wind Speed', 'Ammonia (mg/L)', 'Inorganic Phosphate (mg/L)', 'BOD (mg/l)']
target = 'Phytoplankton (cells/ml)'

# Combine features and target for correlation analysis
df_for_corr = merged_df[features + [target]]

# Calculate correlation matrix
corr_matrix = df_for_corr.corr()

# Print the correlation values with the target variable
target_corr = corr_matrix[target].drop(target)
sorted_target_corr = target_corr.abs().sort_values(ascending=False)

print("Correlation with Phytoplankton (cells/ml):")
print(sorted_target_corr)


# Drop any remaining missing values
merged_df = merged_df.dropna()

# Split data into training and testing sets
X = merged_df[features]
y = merged_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Random Forest
rf_model = RandomForestRegressor()
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f'Random Forest - Mean Squared Error: {mse_rf}')
print(f'Random Forest - Mean Absolute Error: {mae_rf}')
print(f'Random Forest - R^2 Score: {r2_rf}')

# Log hyperparameters and metrics to wandb
wandb.log({
    'MSE': mse_rf,
    'MAE': mae_rf,
    'R2 Score': r2_rf
})

# Finish the wandb run
wandb.finish()


wandb: Currently logged in as: johnwilsonlorin1 (johnwilsonlorin1-laguna-state-polytechnic-university). Use `wandb login --relogin` to force relogin


Correlation with Phytoplankton (cells/ml):
BOD (mg/l)                    0.179928
Ammonia (mg/L)                0.139569
Inorganic Phosphate (mg/L)    0.129177
Humidity                      0.083127
Temperature                   0.058434
Wind Speed                    0.057731
Wind                          0.014732
Name: Phytoplankton (cells/ml), dtype: float64
Random Forest - Mean Squared Error: 5511806085.206495
Random Forest - Mean Absolute Error: 40763.920859307356
Random Forest - R^2 Score: 0.28967823418127736


MAE,▁
MSE,▁
R2 Score,▁
MAE,40763.92086
MSE,5511806085.2065
R2 Score,0.28968
